In [ ]:
import tensorflow as tf
import pathlib
from tensorflow.keras.applications import mobilenet_v2
from keras import Input, Model, layers
from datetime import datetime
import os
import cv2
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import inception_v3
import urllib
import requests

In [ ]:
data_dir = pathlib.Path('drive/My Drive/TIES4911/task4/apparel-images-dataset')
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

11385


In [ ]:
X = []
Y = []

for folder in os.listdir(data_dir):
  labels = folder.split('_')
  for image in os.listdir(os.path.join(data_dir, folder)):
    path_to_image = os.path.join(data_dir, folder, image)
    image = cv2.imread(path_to_image)
    image = cv2.resize(image, (224, 224))

    X.append(image)
    Y.append(labels)

ds_images = np.array(X)
ds_labels = np.array(Y)

In [ ]:
one_hot = MultiLabelBinarizer()
ds_labels_coded = one_hot.fit_transform(ds_labels)
print(one_hot.classes_)
print(ds_labels[0])
print(ds_labels_coded[0])

['black' 'blue' 'brown' 'dress' 'green' 'pants' 'red' 'shirt' 'shoes'
 'shorts' 'white']
['black' 'dress']
[1 0 0 1 0 0 0 0 0 0 0]


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(ds_images, ds_labels_coded, random_state=42, test_size=0.2)

In [ ]:
model = tf.keras.models.load_model("drive/My Drive/TIES4911/task4/task4-4/model")
model.evaluate(x_test, y_test)

72/72 [==============================] - 227s 3s/step - loss: 0.1195 - accuracy: 0.2824


[0.1195276603102684, 0.2823891043663025]

In [ ]:
checkpoint_path = 'drive/My Drive/TIES4911/task4/task4-4/checkpoint-{epoch:04d}'
checkpoint_dir = os.path.dirname(checkpoint_path)
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

drive/My Drive/TIES4911/task4/task4-4/checkpoint-0010


In [ ]:
inputs = Input(shape=(224, 224, 3))

base_model = inception_v3.InceptionV3(input_tensor=inputs, include_top=False)
base_model.trainable = False

preprocess_input = inception_v3.preprocess_input

x = preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(one_hot.classes_), activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv_1 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_1 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten_1 (Flatten)         (None, 51200)             0         
                                                                 
 dense_2 (Dense)             (None, 1024)              5242

In [ ]:
checkpoint_path = 'drive/My Drive/TIES4911/task4/task4-4/checkpoint-{epoch:04d}'
checkpoint_dir = os.path.dirname(checkpoint_path)
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

drive/My Drive/TIES4911/task4/task4-4/checkpoint-0010


In [ ]:
batch_size = 32
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'drive/My Drive/TIES4911/task4/task4-4/logs/%s' % stamp
tensorboard = tf.keras.callbacks.TensorBoard(logdir)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=image_count // batch_size * 2,
    verbose=1)

history = model.fit(x_train, y_train,
                    epochs=20,
                    initial_epoch=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    batch_size=batch_size,
                    callbacks=[tensorboard, model_checkpoint_callback])

NameError: ignored

In [ ]:
model.save("drive/My Drive/TIES4911/task4/task4-4/model")

In [ ]:
classes = one_hot.classes_

urls = [
    'http://picture-cdn.wheretoget.it/e07ql5-l-610x610-dress-little+black+dress-skater+dress-nastygal-deep+vneck-short-formal-short+formal+dress-prom-short+prom+dress-black-lbd-short+black+dress-prom+dress-black+dress-blackdress-short+.jpg',
    'https://d2euz5hho4dp59.cloudfront.net/images/detailed/40/main_jean_419.jpg',
    'https://sc02.alicdn.com/kf/HTB1QbZ_dzgy_uJjSZJnq6zuOXXaq/Wholesale-scratch-pants-damaged-denim-women-s.jpg_350x350.jpg',
    'https://i.ebayimg.com/00/s/NjAwWDYwMA==/z/pakAAOSwVtta6SN8/$_1.JPG?set_id=880000500F',
    'https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSFA1Q-O44dQWt1lvsnOQyoMcQ3myaxY-GscMHgmPtmyWT14ZJU',
    'https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTJYyBOAy35RM7m0JzNGHo_-VTSf6bPMh9hACbhhqxsdoMXHQvD',
    'https://cdn.shopify.com/s/files/1/1359/6121/products/7481209_LF0A0919_1024x1024.jpg?v=1511982241',
]

for url in urls:
  r = requests.get(url, stream = True).raw
  x = np.asarray(bytearray(r.read()), dtype="uint8")
  x = cv2.imdecode(x, cv2.IMREAD_COLOR)
  x = cv2.resize(x, (224, 224))
  x = np.expand_dims(x, axis=0).astype(np.float32)

  preds = model.predict(x)
  top_2 = np.argsort(preds[0])[:-4:-1]
  print(top_2)

  for i in range(2):
    print('{}'.format(classes[top_2[i]] + ' ({:.3})'.format(preds[0][top_2[i]])))

  print()

1/1 [==============================] - 0s 182ms/step
[0 3 1]
black (0.824)
dress (0.176)

1/1 [==============================] - 0s 177ms/step
[5 1 2]
pants (0.981)
blue (0.0191)

1/1 [==============================] - 0s 202ms/step
[5 1 6]
pants (0.964)
blue (0.0214)

1/1 [==============================] - 0s 176ms/step
[7 0 4]
shirt (1.0)
black (9.42e-09)

1/1 [==============================] - 0s 179ms/step
[7 1 4]
shirt (1.0)
blue (5.73e-13)

1/1 [==============================] - 0s 180ms/step
[9 0 1]
shorts (0.967)
black (0.0325)

1/1 [==============================] - 0s 181ms/step
[9 1 4]
shorts (1.0)
blue (0.00011)



In [ ]:
print(np.array(classes))

['black' 'blue' 'brown' 'dress' 'green' 'pants' 'red' 'shirt' 'shoes'
 'shorts' 'white']
